## Post to a LinkedIn Page
This notebook will help you post content to your LinkedIn Organization page.

To-Dos
* Create LinkedIn Organization
* Create an app in LinkedIn Developers
* Get the ClientID and Client Secret
* Get approval from LinkedIn team (2 days)

In [1]:
import requests                 # To deal with REST APIs
import boto3                    # To store AccessKeys in AWS Secret Manager
from config import LinkedIn     # Get configuration details

In [2]:
# Get the ClientID and ClientSecret too get the access token
client_id       = LinkedIn.CLIENT_ID
client_secret   = LinkedIn.CLIENT_SECRET
organization_id = LinkedIn.ORGANIZATION_ID

To receive the Access Token, we need to make two REST API calls:
1. A GET that will redirect you to a new URL. From this URL, you’ll need to extract the value in the parameter code.
1. A POST that will return your brand new Access Token.

https://towardsdatascience.com/linkedin-api-python-programmatically-publishing-d88a03f08ff1

In [3]:
base_url = "https://www.linkedin.com/oauth/v2/authorization"
redirect_uri = "https://datacloudmag.com/auth/linkedin/callback"
scope = "w_member_social,r_liteprofile,rw_organization_admin,r_organization_social,w_organization_social"

params = {
    'response_type' : 'code',
    'client_id' : client_id,
    'redirect_uri' : redirect_uri,
    'state' : 'random',
    'scope' : 'w_member_social,r_liteprofile'
}

# Creating a GET request with the above payload will return an HTML document
# that will ask the user to grant specific permissions to the application.
# So, we need to hit the URL in the browser directly and follow the steps.

url = f"{base_url}?response_type=code&client_id={client_id}&state=random&redirect_uri={redirect_uri}&scope={scope}"
print(url)

https://www.linkedin.com/oauth/v2/authorization?response_type=code&client_id=77r8nkz77j9q3u&state=random&redirect_uri=https://datacloudmag.com/auth/linkedin/callback&scope=w_member_social,r_liteprofile,rw_organization_admin,r_organization_social,w_organization_social


Next Steps:
1. Hit the above URL in the browser.
1. You will be asked to signin with your LinkedIn username and password.
1. Allow the application to access the required information.
1. The response after signing in is below. Extract the **code** from the URL.

In [4]:
response_url = "https://datacloudmag.com/auth/linkedin/callback?code=SOME_RANDOM_CODE&state=random"

auth_code = "SOME_RANDOM_CODE"
auth_code = "AQRzGVoabKv6F038H2R2nHfi8Z7O5iK5ErZuXMJoyqEnICxvNoOEWcWIcAPT6PUqrbs-hQo7-xJpw-KlDcENaMH6vW13gYucCTEnTXgXQWqv9DXdnvhthwBJDou0jMTI3mhoUu1jquzFBZy1EoLNHINJehvIdGIo2l95-fSD2vYE4SLnCjSQT7PLofTJsDxKjUNCrCkF9_401ha-rw0"

### Get Access Token using code from previous step
Now, using the **code** from the previous step, make the POST request.

In [5]:
url_access_token = "https://www.linkedin.com/oauth/v2/accessToken"

payload = {
    'grant_type' : 'authorization_code',
    'code' : auth_code,
    'redirect_uri' : redirect_uri,
    'client_id' : client_id,
    'client_secret' : client_secret
}

response = requests.post(url=url_access_token, params=payload)

response_json = response.json()

print(response_json)

{'access_token': 'AQV9820MbGqrmRUbZh7CP9VqlFRfMn3VNg9Qw4KkXCpaP8Z8vboFVEU4cY1fS-lr_9yOWbb8O9z9AaVbDKGCdFZWV32W9_PHN95W36QDIomPZ4SXhwmN81r4ZsB9rV5YqIpdHiwV5uO2QcunHKxXMI6yDxbaShSc4hLWgKhKgbzL1zCvuVlFIYiQBxUofHD4aKHxX2GvgOVbDMJC4DS7zqueyQkwoFM1OaBC11HWzNugICKD7lD3lpM5FzLRc3drxt5gEVCLIyk0nLl0aEsFsuziKI0s2DAJDQPrOf8jzKOwhAJXfm8fxJzVNGbfQILtFEmEAs2wAC9FdTnfKQ70HBlfC4OYQg', 'expires_in': 5183999, 'refresh_token': 'AQVGG-mw2XLux5uAUsmHRZ8bkul4WN8uXi4vaI3FJ_U-KNKleBdxf3k8aFF6EcT5kvg_PsL4HOQbJVxXNcoEyMa-MQrf7q8qPwCIyrF4ij6C4h1dF8bko_vRuzO8ifM_ziMwLuwocCXh5XIJGewGAorLc93h5dkcZzkLcbYT6hWHr-WZYfdww2cvDle6389FARoaPZkxrobkSVqNSWAC4ySHRyg4YBFWN1HWT53d_9C5xLJpOdIx2xJu6uXuThMd6jWsJX5xsWZxHTFXXG2daF3hFJPQqcWIa2aVSXxcieB4M2EidBsTX1BZOQfIVCoOmgW7Nsr6-jPw0j0eZwRTQuZDiVMMLQ', 'refresh_token_expires_in': 31535999}


In [6]:
# Extract the access_token from the response_json
access_token = response_json['access_token']

### Share the post on LinkedIn Page using the Access Token obtained.

In [7]:
# Share post on LinkedIn

# https://docs.microsoft.com/en-us/linkedin/marketing/integrations/community-management/shares/share-api?tabs=http

url = "https://api.linkedin.com/v2/shares"

headers = {
    'Authorization' : f'Bearer {access_token}',
    'Content-Type' : 'application/json'
}

payload = {
    "content": {
        "contentEntities": [
            {
                "entityLocation": "https://datacloudmag.com/solving-the-gilded-rose-refactoring-kata/",
                "thumbnails": [
                    {
                        "resolvedUrl": "https://i0.wp.com/datacloudmag.com/wp-content/uploads/2022/02/pexels-photo-169573.jpeg?"
                    }
                ]
            }
        ],
        "title": "Test Share with Content"
    },
    'distribution': {
        'linkedInDistributionTarget': {}
    },
    'owner': f'urn:li:organization:{organization_id}',
    'text': {
        'text': f'Code Refactoring by Aveek Das - Solving the Gilded Rose Refactoring Kata \n#python #sql #refactoring',
        'annotations': [
            {
                'entity' : f'urn:li:person:G0CopwI-SP',
                'length' : 9,
                'start' : 20
            }
        ]
    }
}

response = requests.post(url=url, headers=headers, json = payload)

print(response.json())

{'owner': 'urn:li:organization:79377523', 'activity': 'urn:li:activity:6908166778082746368', 'edited': False, 'created': {'actor': 'urn:li:person:G0CopwI-SP', 'time': 1647035307412}, 'text': {'annotations': [{'length': 9, 'start': 20, 'entity': 'urn:li:person:G0CopwI-SP'}], 'text': 'Code Refactoring by Aveek Das - Solving the Gilded Rose Refactoring Kata \n#python #sql #refactoring'}, 'lastModified': {'actor': 'urn:li:person:G0CopwI-SP', 'time': 1647035307412}, 'id': '6908166777478778880', 'distribution': {'linkedInDistributionTarget': {'visibleToGuest': True}}, 'content': {'title': 'Test Share with Content', 'contentEntities': [{'thumbnails': [{'imageSpecificContent': {}, 'resolvedUrl': 'https://i0.wp.com/datacloudmag.com/wp-content/uploads/2022/02/pexels-photo-169573.jpeg?'}], 'title': 'Test Share with Content', 'entityLocation': 'https://datacloudmag.com/solving-the-gilded-rose-refactoring-kata/'}], 'shareMediaCategory': 'ARTICLE'}}


In [ ]:
# Get LinkedIn user ID

url = "https://api.linkedin.com/v2/me"

header = {
    'Authorization' : f'Bearer {access_token}'
}

r = requests.get(url=url, headers=header)

print(r.text)

In [ ]:
# Get LinkedIn Access token from AWS SSM

client = boto3.client('ssm')

param = client.get_parameter(
    Name = "/social_integration/linkedin/access_token",
    WithDecryption = False
)

print(param['Parameter']['Value'])

In [ ]:
text = "In this article, Aveek Das describes how to refactor code."
username = "Aveek Das"

# start = 32
# length = 9

print(text.find(username))
print(len(username))